In [ ]:
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()

In [ ]:
df = pd.read_csv("http://data.insideairbnb.com/canada/bc/vancouver/2023-09-06/data/listings.csv")

In [ ]:
van_neighbourhoods_map_url = "http://data.insideairbnb.com/canada/bc/vancouver/2023-09-06/visualisations/neighbourhoods.geojson"
map_data = alt.Data(url=van_neighbourhoods_map_url, format=alt.DataFormat(property='features',type='json'))

In [ ]:
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)
review_cols = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness','review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value']
df = df.melt(id_vars=df.columns.difference(review_cols), var_name='review_type', value_name='rating')
df.shape

In [14]:
map_summary_data = df[["neighbourhood_cleansed", "availability_365", "price"]].groupby('neighbourhood_cleansed').agg(
  mean_availability_365=('availability_365', 'mean'),
  listing_count=('neighbourhood_cleansed', 'size'),
  avg_price=('price', 'mean')
).reset_index()

In [ ]:
brush_neighbourhood = alt.selection_point(on='click', fields=['properties.neighbourhood'], empty = False)

basemap = alt.Chart(map_data).mark_geoshape(
  stroke='white',
  strokeWidth=1.5
).transform_lookup(
  lookup='properties.neighbourhood',
  from_=alt.LookupData(map_summary_data, "neighbourhood_cleansed", ['mean_availability_365', 'listing_count', 'avg_price'])
).encode(
  # color=alt.condition(brush_neighbourhood, alt.value('red'), 'avg_price:Q'),
  color='avg_price:Q',
  tooltip=['properties.neighbourhood:N', 'mean_availability_365:Q', 'listing_count:Q', 'avg_price:Q']
).add_params(
    brush_neighbourhood
)


In [ ]:
brush_listings = alt.selection_interval(encodings=['longitude', 'latitude'], empty = True)

listings = alt.Chart(df).mark_circle(
    size = 8,
    opacity = 0.7,
    color = 'red'
).encode(
    longitude="longitude:Q", 
    latitude="latitude:Q",
    # color = alt.condition(brush_listings, alt.value('red'), alt.value('lightgray')),
     tooltip=['name', 'description']
).add_params(
    brush_listings
).transform_filter(
    brush_listings
)

In [ ]:
brush_price = alt.selection_interval(encodings=['x'])

price_tick = alt.Chart(
    data = df,
    width = 700
).add_params(
    brush_price
).mark_tick(
).encode(
    x = alt.X('price', scale=alt.Scale(domain = [0,2300])).axis(
        title = 'Price per Night',
        orient='top',
        ticks=False,
        domain=True
    ),
    color=alt.condition(
        brush_price,
        alt.value('steelblue'),
        alt.value('lightgrey'))
).transform_filter(
    alt.datum.price <= 2300
)

In [ ]:
review_scores = alt.Chart(df).mark_boxplot(
).encode(
    y='review_type:N',
    x='rating:Q'
)

In [ ]:
piechart_hostIsSuperhost = alt.Chart(df).mark_arc(size=100).encode(
    theta='count()',
    color='host_is_superhost',
    tooltip=['count()']
).transform_filter(
    alt.datum.host_is_superhost != None
)
piechart_instantBookable = alt.Chart(df).mark_arc(size=100).encode(
    theta='count()',
    color='instant_bookable',
    tooltip=['count()']
)
piechart_propertyType = alt.Chart(df).mark_arc(size=100).encode(
    theta='count()',
    color='property_type',
    tooltip=['count()','room_type','property_type']
)

pies = alt.vconcat(piechart_hostIsSuperhost, piechart_instantBookable)

In [ ]:
bar_propertyType = alt.Chart(df).mark_bar().transform_aggregate(
    property_count='count()',
    groupby=['property_type']
).transform_window(
    rank='rank(property_count)',
    sort=[alt.SortField('property_count', order='descending')]
).encode(
    y=alt.Y('property_count:Q', title='Count'),
    x=alt.X('property_type:N').sort('-y'),
    tooltip=['property_count:Q', 'property_type']
).transform_filter(
    (alt.datum.rank <= 10)
)

In [ ]:
ava_days = alt.Chart(df).mark_bar().encode(
    alt.X('availability_365',axis=alt.Axis(title='Number of Days Available'),scale=alt.Scale(domain=[0, 365])),
    alt.Y('count()',axis=alt.Axis(title='Number of Listings')),
    tooltip=['neighbourhood_cleansed','count()']
).properties(
    title='Nubmer of listing vs. Number of Days Available'
)

In [ ]:
# vancouver_map

selected_price_tick = price_tick.transform_filter(brush_listings)

selected_ava_days = ava_days.transform_filter(brush_price).transform_filter(brush_listings)
selected_pies = pies.transform_filter(brush_price).transform_filter(brush_listings)
selected_review_scores = review_scores.transform_filter(brush_price).transform_filter(brush_listings)
selected_bar_propertyType = bar_propertyType.transform_filter(brush_price).transform_filter(brush_listings)

listings_selected = listings.encode(
    color = alt.condition(brush_price, alt.value('red'), alt.value('grey')),
    opacity = alt.condition(brush_price, alt.value(1), alt.value(0.2))
)

vancouver_map = (basemap + listings_selected).project(
    type='identity', reflectY=True
).properties(
    width = 800,
    height = 800
)

row = selected_ava_days | selected_bar_propertyType  | selected_pies | selected_review_scores

# boxplot_row

dashboard = alt.vconcat(vancouver_map, selected_price_tick, row)

dashboard

